In [1]:
import pandas as pd
import numpy as np
import re
import json
import pprint

In [2]:
data = pd.read_csv('./wuzzuf-data/data_science_dataset_wuzzuf.csv')

In [60]:
# cleaning The "city_name" column 
# my target is to add each city to same name.
# print the unique city names entered.



unique_cities = np.unique(data.city_name)
print "** there are ", len(np.unique(data.city_name)), "unique city name"
with open('./munging-data/unique_cities_unexpanded','wb') as f:
    for item in np.unique(data.city_name):
        f.write(item+"\n")

value_counts = data.city_name.value_counts()
unique_above_x = value_counts[value_counts>=3].keys()

unique_extracted = np.unique([ city.lower().strip() for city in unique_above_x ]) 
unique_extracted = [ re.split('[.,/&\-]|\sand\s|\sor\s',city,re.UNICODE) for city in unique_extracted ]

unique_list = [ s.strip() for item in unique_extracted for s in item  ]
unique_list = np.unique(unique_list)

with open('./munging-data/unique_cities_expanded','wb') as f:
    for item in unique_list:
        f.write(item+'\n')

english_pattern = re.compile('^[\d\w\s]*$')
unique_english_list = []
unique_arabic_list = []
for city in unique_list:
    if english_pattern.match(city) :
        unique_english_list.append(city)
    else:
        unique_arabic_list.append(city)  

print "** ", len(unique_list), " unique city names after filteration"
print "** ", len(unique_arabic_list), " Arabic city names"
print "** ", len(unique_english_list), " English city names"        

** there are  478 unique city name
**  104  unique city names after filteration
**  6  Arabic city names
**  98  English city names


In [62]:
with open('./munging-data/cities_names_dict.json','rb') as f:
    cities_dict = json.load(f)

In [61]:
def clean_city_name(d):
    city_name = d.city_name.lower().strip()
    cities = re.split('[.,/&\-]|\sand\s|\sor\s',city_name)
    new_cities = []
    for c in cities :
        if c == "": break;
        flag = True
        for item in cities_dict :
            for ii in cities_dict[item] :
                if re.search(ii, c) :
                    new_cities.append(item);
                    flag = False;
                    break
        if flag == True :
            new_cities.append("other('"+c.lower().strip()+"')")
    
    cities_str = ""
    for c in new_cities :
        if cities_str == "" : 
            cities_str += c
        else : 
            cities_str = cities_str +", "+c
    d.city_name = cities_str
    return d
                
new_data = data.apply(clean_city_name, axis=1)
# 297 others, and 21,533 checked

In [63]:
df2 = pd.DataFrame(columns=new_data.columns)
for x in new_data.iterrows() :
    index = x[0]
    row = x[1]
    if re.search(", ", row.city_name):
        df2.loc[len(df2)] = row
        new_data.drop(index, inplace=True)


In [64]:
indexend = len(df2)
for x in df2.iterrows():
    index = x[0]
    row = x[1]
    cities = row.city_name.split(', ')
    for city in cities :
        alt = row
        alt.city_name = city
        df2.loc[indexend] = alt
        indexend += 1 
    df2.drop(index,inplace=True)
        

In [65]:
df2.reset_index(drop=True, inplace=True)
new_data.reset_index(drop=True, inplace=True)

index = len(new_data)
for x in df2.iterrows():
    row = x[1]
    new_data.loc[index] = row
    index += 1;

In [66]:
data_otherCities_filtered = new_data.copy()
for x in data_otherCities_filtered.iterrows():
    index, row = x
    if re.search('other', row.city_name):
        data_otherCities_filtered.drop(index, inplace=True)
data_otherCities_filtered.reset_index(drop=True, inplace=True)
data_otherCities_filtered.to_csv('./wuzzuf-data/wuzzuf_data_with_cityname_cleanedandfiltered.csv',index=False)